In [92]:
import pandas as pd
import os

In [93]:
ROOT_DIR = "D:\ML_Projects\cancer_prediction"
DATA_DIR = os.path.join(ROOT_DIR, "data")
SAVED_MODEL_DIR = os.path.join(ROOT_DIR, "models")
dataset_path = os.path.join(DATA_DIR,'breast-cancer_02.csv')

cancer_df = pd.read_csv(dataset_path)


<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\eron2\AppData\Local\Temp\ipykernel_18876\2588803651.py:1: SyntaxWarning: invalid escape sequence '\M'
  ROOT_DIR = "D:\ML_Projects\cancer_prediction"


In [94]:
X = cancer_df[['perimeter_mean','area_mean', 'concavity_mean','compactness_mean','concave points_mean', 'radius_worst', 'area_worst', 'perimeter_worst','concavity_worst', 'concave points_worst']]
y_reg = cancer_df['radius_mean']
y_clf = cancer_df['diagnosis']

y_clf.value_counts()

diagnosis
0    357
1    212
Name: count, dtype: int64

In [95]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled=scaler.fit_transform(X)
X_scaled[:5]

array([[ 1.26993369,  0.9843749 ,  2.65287398,  3.28351467,  2.53247522,
         1.88668963,  2.00123749,  2.30360062,  2.10952635,  2.29607613],
       [ 1.68595471,  1.90870825, -0.02384586, -0.48707167,  0.54814416,
         1.80592744,  1.89048899,  1.53512599, -0.14674897,  1.0870843 ],
       [ 1.56650313,  1.55888363,  1.36347845,  1.05292554,  2.03723076,
         1.51187025,  1.45628455,  1.34747521,  0.85497394,  1.95500035],
       [-0.59268717, -0.76446379,  1.91589718,  3.40290899,  1.45170736,
        -0.28146446, -0.55002123, -0.2499393 ,  1.98958826,  2.17578601],
       [ 1.77657315,  1.82622928,  1.37101143,  0.53934045,  1.42849277,
         1.29857524,  1.22072425,  1.33853946,  0.61317876,  0.72925926]])

In [96]:
import joblib
joblib.dump(scaler,os.path.join(SAVED_MODEL_DIR,'scaler.pkl'))

['D:\\ML_Projects\\cancer_prediction\\models\\scaler.pkl']

In [97]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train_reg,y_test_reg = train_test_split(X_scaled,y_reg,test_size=0.2,random_state=42)

In [98]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train,y_train_reg)
joblib.dump(linear_model,os.path.join(SAVED_MODEL_DIR,'linear_model.pkl'))

['D:\\ML_Projects\\cancer_prediction\\models\\linear_model.pkl']

In [99]:
from sklearn.ensemble import RandomForestRegressor

raf_regressor = RandomForestRegressor(n_estimators=100,random_state=42)
raf_regressor.fit(X_train,y_train_reg)
joblib.dump(raf_regressor,os.path.join(SAVED_MODEL_DIR,'raf_regressor.pkl'))

['D:\\ML_Projects\\cancer_prediction\\models\\raf_regressor.pkl']

In [100]:
reg_models = {'linear_model':linear_model,'regressor':raf_regressor}


In [101]:
from sklearn.metrics import mean_squared_error
for name, model in reg_models.items():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test_reg, y_pred)
    print(f"{name} - MSE: {mse:.2f}")

linear_model - MSE: 0.01
regressor - MSE: 0.02


In [ ]:
X_with_radius = X.copy()
# X_with_radius['pred_with_radius'] = linear_model.predict(X_scaled)
# X_with_radius['pred_with_radius_02'] = raf_regressor.predict(X_scaled)
X_with_radius['pred_radius'] = (linear_model.predict(X_scaled) + raf_regressor.predict(X_scaled))/2

scaler_clf = StandardScaler()
X_clf_scaled = scaler_clf.fit_transform(X_with_radius)
joblib.dump(scaler_clf, os.path.join(SAVED_MODEL_DIR,'scaler_clf.pkl'))

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X_train,X_test,y_train_clf,y_test_clf = train_test_split(X_clf_scaled,y_clf,test_size=0.2,random_state=42)

smote = SMOTE(random_state=42)
X_resample,y_resample = smote.fit_resample(X_train,y_train_clf)

(pd.Series(y_resample).value_counts())
(pd.Series(y_train_clf).value_counts())



array([[ 1.26993369,  0.9843749 ,  2.65287398, ...,  2.10952635,
         2.29607613,  1.07537593],
       [ 1.68595471,  1.90870825, -0.02384586, ..., -0.14674897,
         1.0870843 ,  1.82532604],
       [ 1.56650313,  1.55888363,  1.36347845, ...,  0.85497394,
         1.95500035,  1.58252826],
       ...,
       [ 0.67267578,  0.57795264,  0.04658753, ...,  0.3267666 ,
         0.41406869,  0.70582933],
       [ 1.98252415,  1.73521799,  3.296944  , ...,  3.19760468,
         2.28998549,  1.84691135],
       [-1.81438851, -1.34778924, -1.11487284, ..., -1.30583065,
        -1.74506282, -1.77766532]], shape=(569, 11))

In [103]:
from sklearn.linear_model import LogisticRegression

logic_clf = LogisticRegression(random_state=42)
logic_clf.fit(X_resample,y_resample)
joblib.dump(logic_clf,os.path.join(SAVED_MODEL_DIR,'logic_clf.pkl'))

['D:\\ML_Projects\\cancer_prediction\\models\\logic_clf.pkl']

In [104]:
from sklearn.metrics import accuracy_score,classification_report
y_pred = logic_clf.predict(X_test)
acc = accuracy_score(y_test_clf,y_pred)
report=classification_report(y_test_clf,y_pred)
print(acc)
print(report)

0.9824561403508771
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        71
           1       0.96      1.00      0.98        43

    accuracy                           0.98       114
   macro avg       0.98      0.99      0.98       114
weighted avg       0.98      0.98      0.98       114

